# Load SLEGO and Import Libraries

In [10]:
#@title
%pip install jupyter_bokeh
%pip install panel
import requests
import json
import sys
import panel as pn
import pandas as pd
from io import StringIO
import inspect
import time
import param
from collections import ChainMap
import os

from IPython.display import clear_output

pn.extension(sizing_mode = 'stretch_width')
pn.extension('plotly')

class FuncSelector(param.Parameterized):
    def __init__(self, module, datapath='./data/', tag='first'):
        self.tag=tag
        self.data_format = [pd.DataFrame ] #['_data']
        self.messagebox = pn.widgets.TextInput( name='Message', placeholder='1 select a function, 2 input parameters , 3 click compute button')
        self.datapath = datapath
        self.module = module
        self.funcs_dict, self.funcs_name_list, self.funcs_argument_list = self.get_function_name() #+type list
        self.btn_compute = pn.widgets.Button(name='Compute Block', button_type = 'primary')
        self.sel_func = pn.widgets.MultiChoice(name='Select an analytic service:', options=['']+self.funcs_name_list, max_items =1)
        self.card_param = pn.Card(title='Input parameters:')
        self.sel_func.param.watch(self.select_func_change, 'value')
        self.btn_compute.on_click(self.btn_compute_clicked)
        self.func_name = None
        self.func_param_dict = None
        self.func = None
        self.data_list = None
        self.func_param_dict = [] #!!
        self.result = None
        self.display = pn.Column()
        self.text_input_save = pn.widgets.TextInput(name='output data names' ,placeholder='result_functionName_[current_time].csv')
        self.param_type = {}
        self.data_param_list = []
        self.func_desc = pn.widgets.TextAreaInput(name='Instruction' ,height_policy='max')
        self.msgbox_result = pn.widgets.TextAreaInput( name='Result', placeholder='Display result after runiing service',height_policy='max',   min_height=250)
        
    def btn_compute_clicked(self,event):
        self.msgbox_result.value=''
        self.btn_compute.button_type = 'warning'
        self.display.clear()
        self.messagebox.placeholder = 'Computing...please wait'
        start_time = time.time()
        self.result = self.func(**self.func_param_dict)
        exec_time = time.time() - start_time
        self.messagebox.placeholder = 'Task finished with time: '+str( round(exec_time,3))+ ' seconds.'
        self.btn_compute.button_type = 'success'
        self.msgbox_result.value = str(self.result)
    
    def select_func_change(self,event):
        self.func_desc.value=''
        self.display.clear()
        self.btn_compute.button_type = 'primary'
        self.card_param.clear()
        if self.sel_func.value == []:
            return
        self.func_value = self.sel_func.value[-1]
        self.func_name = self.func_value 
        self.func = eval('self.module.'+self.func_name)
        self.param_type = inspect.getfullargspec(self.func)[6] 
        self.func_param_dict = self.funcs_dict[self.func_value ]
        self.param_inputs = []    

        for k,v in self.func_param_dict.items():
            arg_input = pn.widgets.LiteralInput(name=k, value=v)
            arg_input.param.watch(self.text_changed,'value')
            self.param_inputs.append(arg_input)
        self.card_param.objects = self.param_inputs
        self.func_desc.value = self.func.__doc__


    def text_changed(self,event):
        self.btn_compute.button_type = 'primary'
        #https://panel.holoviz.org/user_guide/Links.html?highlight=event
        self.func_param_dict[event.obj.name] = event.obj.value

    def get_function_name(self):
        funcs = inspect.getmembers(self.module, inspect.isfunction)
        funcs_name_list = [fun[0] for fun in funcs]
        funcs_argument_list = [inspect.getfullargspec(fun[1])[0]  for fun in funcs]
        funcs_argument_default_list = [inspect.getfullargspec(fun[1])[3]  for fun in funcs]
        funcs_dict = [{f:dict(zip(a,d))} for f,a,d in zip(funcs_name_list, funcs_argument_list, funcs_argument_default_list)]
        funcs_dict =dict(ChainMap(*funcs_dict ))
        return funcs_dict, funcs_name_list, funcs_argument_list #+type list

    @property
    def view(self):
        return pn.Column(pn.Row( 
            pn.Column(self.sel_func,pn.Row(self.messagebox),self.btn_compute,self.msgbox_result),
            self.card_param,
            self.func_desc))

clear_output()

# Import functions

In [11]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objs as go
from jsonschema import validate
import json
import json
import pandas as pd
from datetime import datetime
import plotly.graph_objects as go


def adage3_json_schema_validation(filepath:str= 'jsonfile_quote.json'):
    from jsonschema import validate
    import json
    '''
    input the file name and validate the data
    '''
    f = open(filepath)
    data = json.load(f)

    schema = {
    "type": "object",
    "properties": 
    {
        "data_source": {"type": "string"},
        "dataset_type_id": {"type": "string"},
        
        "time_object": {
        "type": "object",
        "properties": {
            "timestamp": {"type": "string"},
            "timezone": {"type": "string"}},
        "required": ["timestamp","timezone"]
                        },
        "events": {"type": "array",
                "items": {"$ref":"#/$defs/events"},
                "minItems": 1,
        }
    },

        "required": ["data_source","dataset_type_id","time_object","events"],

        "$defs": {
            "events": {
                "type": "object",
                "properties": {
                    "time_object": {"type": "object",
                                    "properties": {
                                        "timestamp":{"type": "string"},
                                        "duration":{"type": "number"},
                                        "timezone":{"type": "string"},
                                                    },
                    
                                    "required": ["timestamp","duration","timezone"],       
                                    },
                        
                    "event_type": {"type": "string"},
                    "attribute": {"type": "object"},
                    },
                "required": ["time_object","event_type","attribute"],
                },
        }
    }
    validate(instance=data,schema= schema)
    return 'Valid data'


def convert_adage3_attribute_json2csv(filepath:str='jsonfile_ohlc.json', csv_name:str='json2csv'):
    import json
    import pandas as pd
    from datetime import datetime
    '''
    file will be saved in current repository
    '''


    f = open(filepath)
    data = json.load(f)

    list_att=[]
    list_time=[]

    for i in range(len(data['events'])):
        list_att.append(data['events'][i]['attribute'])
        list_time.append(data['events'][i]['time_object']['timestamp'])


    # Convert the list to a pandas DataFrame
    df = pd.DataFrame(list_att)

    timestamp_strings =list_time
    timestamp_format = "%Y-%m-%d %H:%M:%S.%f"
    #timestamps = [datetime.strptime(ts, timestamp_format) for ts in timestamp_strings]
    try:
        timestamps = [datetime.strptime(ts, timestamp_format) for ts in timestamp_strings]
    except:
        timestamps = timestamp_strings    
    df.index = timestamps

  
    df.to_csv(csv_name+'.csv')

    return df

def visualize_adage3_csv(filepath='data.csv', output_html_name='adage3_data_viz'):
    from pandas.api.types import is_number
    '''
    Result will be saved in current path as a csv file 
    '''
    import plotly.express as px
    import pandas as pd
    from plotly.subplots import make_subplots
    import plotly.graph_objs as go

    df = pd.read_csv(filepath, index_col=0)
    timestamps = df.index
    df['timestamps'] =timestamps
    df.insert(0, 'timestamps', df.pop('timestamps'))
    col_names = list(df.columns)
    coltypes = list(df.dtypes)

    specs =[[{"type": "table"}]]

    for col in col_names:
        specs.append([{"type": "scatter"}])

    fig = make_subplots(
        rows=df.shape[1]+1, cols=1,
        shared_xaxes=False,
        vertical_spacing=0.03,
        subplot_titles=['timestamps']+col_names,
        specs=specs)

    fig.add_trace(go.Table(header=dict(values=col_names,font=dict(size=10),align="left"),
            cells=dict(values=[df[k].tolist() for k in df.columns],align = "left")),row=1, col=1)

    for i in range(len(col_names)):
        if coltypes[i]== 'O':
        
            y_out= pd.to_numeric(df[col_names[i]], errors='ignore', downcast=None)
            if not is_number(y_out):
                y_out= df[col_names[i]].apply(len)
            #fig.add_scatter(x=timestamps, y=y_out, hovertemplate=df[col_names[i]],  row=1+i, col=1, name=col_names[i])
            fig.add_trace(go.Scatter(x=timestamps, y=y_out, hovertemplate=df[col_names[i]], name=col_names[i]), row=2+i, col=1)
        else:
            #fig.add_scatter(x=timestamps, y=df[col_names[i]], row=1+i, col=1, name=col_names[i])
            fig.add_trace(go.Scatter(x=timestamps, y=df[col_names[i]],  name=col_names[i]), row=2+i, col=1)

    # Show the plot
    fig.update_layout(height=250*len(col_names), title_text="Visalized ADAGE3 data" ,legend=dict(orientation="h" ))
    fig.write_html(output_html_name+".html")

    return df
    

def visualise_adage3_json(filepath:str='jsonfile_ohlc.json',output_html_name='adage3_data_viz'):
    '''
    Result will be saved in current path as a html file 
    '''

    convert_adage3_attribute_json2csv(filepath=filepath, csv_name='temp' )
    result = visualize_adage3_csv(filepath='temp.csv', output_html_name=output_html_name)

    return result

def validate_and_visualise_adage3_json(filepath:str='jsonfile_ohlc.json',output_html_name='adage3_data_viz'):
    '''
    Result will be saved in current path as a html file 
    '''
    
    adage3_json_schema_validation(filepath= filepath)
    convert_adage3_attribute_json2csv(filepath=filepath, csv_name='temp' )
    result = visualize_adage3_csv(filepath='temp.csv', output_html_name=output_html_name)

    return result

#===========================================create function file=================================
filename = "func.py"

if os.path.exists(filename):
    os.remove(filename)
    print(f"File '{filename}' was removed.")
else:
    print(f"File '{filename}' does not exist.")


print(f'{filename} is being created. Here are the functions:')
for name, obj in inspect.getmembers(sys.modules[__name__]):
    if inspect.isfunction(obj) and name != 'clear_output' and name != 'open'and name != 'make_subplots':
      with open(filename, "a+") as f:
          print(name)
          f.write(inspect.getsource(obj))
          f.write("\n")


sys.path.insert(0,'filename')

File 'func.py' was removed.
func.py is being created. Here are the functions:
adage3_json_schema_validation
convert_adage3_attribute_json2csv
obj
validate
validate_and_visualise_adage3_json
visualise_adage3_json
visualize_adage3_csv


# Launch application

In [18]:
import func
import importlib
importlib.reload(func)
import sys

if 'google.colab' in sys.modules:
    print('Running on Google Colab')
    pn.extension(comms='colab')
else:
    pn.extension(comms='vscode')


app1 = FuncSelector(func)
app1.view

BokehModel(combine_events=True, render_bundle={'docs_json': {'01f7c450-10b2-4e78-85be-c6d4125f0a74': {'defs': …

In [16]:
sys.argv[0]

'/home/codespace/.local/lib/python3.10/site-packages/ipykernel_launcher.py'